In [25]:
import random
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import Dict, List
from transformers import AutoTokenizer
from tqdm import tqdm
import scml
from scml import nlp as snlp
import mylib

In [26]:
percentiles = [.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()

In [27]:
pretrained_dir = "pretrained/google/electra-small-discriminator"
model_max_length = 512
tokenizer = AutoTokenizer.from_pretrained(pretrained_dir, model_max_length=model_max_length)
print(f"{repr(tokenizer)}\n{tokenizer.model_input_names}")

PreTrainedTokenizerFast(name_or_path='pretrained/google/electra-small-discriminator', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
['input_ids', 'token_type_ids', 'attention_mask']


In [28]:
train = pd.read_csv("input/train.csv", engine="c", low_memory=False)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19661 entries, 0 to 19660
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Id             19661 non-null  object
 1   pub_title      19661 non-null  object
 2   dataset_title  19661 non-null  object
 3   dataset_label  19661 non-null  object
 4   cleaned_label  19661 non-null  object
dtypes: object(5)
memory usage: 768.1+ KB


In [29]:
gt_map: Dict[str, List[str]] = {}
dl_map: Dict[str, List[str]] = {}
for t in tqdm(train.itertuples()):
    pid = getattr(t, "Id")
    cl = getattr(t, "cleaned_label")
    dl = getattr(t, "dataset_label")
    if pid not in gt_map:
        gt_map[pid] = list()
    if pid not in dl_map:
        dl_map[pid] = list()
    gt_map[pid].append(cl)
    dl_map[pid].append(dl)

19661it [00:00, 327397.43it/s]


In [30]:
rows = []
sep = "|"
m = 1000
max_negative_examples = 20
max_length = 502
step = 50
for rid, gts in tqdm(gt_map.items()):
    is_multi = 0
    if len(gts) > 1:
        is_multi = 1
    ground_truth = sep.join(sorted(gts))
    dataset_labels = sep.join(sorted(dl_map[rid]))
    with open(f"input/train/{rid}.json") as in_file:
        sections = json.load(in_file)
    tmp = []
    for section in sections:
        tmp.append(section["text"])
    text = " ".join(tmp).strip()
    text = snlp.to_ascii_str(text)
    spans = []
    for dl in dl_map[rid]:
        i = 0
        j = len(text)
        while i < j - len(dl) + 1:
            p = text.find(dl, i, j)
            if p == -1:
                break
            q = p + len(dl)
            spans.append((p, q))
            i = q
            # Right context: answer is on the left edge of window
            a = p
            b = 2 * m + a
            _len = len(tokenizer(text[a:b])["input_ids"])
            while _len > max_length and a < b:
                b -= step * 2
                _len = len(tokenizer(text[a:b])["input_ids"])
            row = {}
            row["Id"] = rid
            row["is_multi"] = is_multi
            row["ground_truth"] = ground_truth
            row["dataset_labels"] = dataset_labels
            row["is_impossible"] = 0
            row["answer_start"] = 0
            row["answer_end"] = len(dl)
            row["context"] = text[a:b]
            row["context_token_length"] = _len
            rows.append(row)
            # Left context: answer is on the right edge of window
            b = q
            a = max(0, b - (2 * m))
            _len = len(tokenizer(text[a:b])["input_ids"])
            while _len > max_length and a < b:
                a += step * 2
                _len = len(tokenizer(text[a:b])["input_ids"])
            row = {}
            row["Id"] = rid
            row["is_multi"] = is_multi
            row["ground_truth"] = ground_truth
            row["dataset_labels"] = dataset_labels
            row["is_impossible"] = 0
            row["answer_end"] = b - a
            row["answer_start"] = row["answer_end"] - len(dl) 
            row["context"] = text[a:b]
            row["context_token_length"] = _len
            rows.append(row)
            # Left and right context: answer is in the middle of window
            a = max(0, p - m)
            b = q + m
            _len = len(tokenizer(text[a:b])["input_ids"])
            while _len > max_length and a < b:
                a += step
                b -= step
                _len = len(tokenizer(text[a:b])["input_ids"])
            row = {}
            row["Id"] = rid
            row["is_multi"] = is_multi
            row["ground_truth"] = ground_truth
            row["dataset_labels"] = dataset_labels
            row["is_impossible"] = 0
            row["context"] = text[a:b]
            row["answer_start"] = row["context"].index(dl)
            row["answer_end"] = row["answer_start"] + len(dl)
            row["context_token_length"] = _len
            rows.append(row)
    i = 0
    negative_examples = int(len(text) / (2 * m)) - len(spans)
    negative_examples = min(max_negative_examples, negative_examples)
    while i < negative_examples:
        b = len(text) - 1 - (2 * m)
        a = random.randint(0, b)
        b = 2 * m + a
        is_overlap = False
        for span in spans:
            if a <= span[0] <= b or a <= span[1] <= b:
                is_overlap = True
                break
        if is_overlap:
            continue
        _len = len(tokenizer(text[a:b])["input_ids"])
        while _len > max_length and a < b:
            a += step
            b -= step
            _len = len(tokenizer(text[a:b])["input_ids"])
        row = {}
        row["Id"] = rid
        row["is_multi"] = is_multi
        row["ground_truth"] = ground_truth
        row["dataset_labels"] = dataset_labels
        row["is_impossible"] = 1
        row["context"] = text[a:b]
        row["answer_start"] = -1
        row["answer_end"] = -1
        row["context_token_length"] = _len
        rows.append(row)
        i += 1

100%|██████████| 14316/14316 [08:08<00:00, 29.33it/s] 


In [31]:
df = pd.DataFrame.from_records(rows)
cols = ["is_multi", "is_impossible"]
df[cols] = df[cols].astype(np.int8)
cols = ["answer_start", "answer_end", "context_token_length"]
df[cols] = df[cols].astype(np.int16) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360000 entries, 0 to 359999
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   Id                    360000 non-null  object
 1   is_multi              360000 non-null  int8  
 2   ground_truth          360000 non-null  object
 3   dataset_labels        360000 non-null  object
 4   is_impossible         360000 non-null  int8  
 5   answer_start          360000 non-null  int16 
 6   answer_end            360000 non-null  int16 
 7   context               360000 non-null  object
 8   context_token_length  360000 non-null  int16 
dtypes: int16(3), int8(2), object(4)
memory usage: 13.7+ MB


In [32]:
df["is_multi"].value_counts(normalize=True)

0    0.524258
1    0.475742
Name: is_multi, dtype: float64

In [33]:
df["is_impossible"].value_counts(normalize=True)

0    0.50835
1    0.49165
Name: is_impossible, dtype: float64

In [34]:
assert (df["context_token_length"] <= max_length).all()
df["context_token_length"].describe(percentiles=percentiles)

count    360000.000000
mean        418.751139
std          58.558810
min           4.000000
1%          134.000000
5%          353.000000
10%         370.000000
20%         387.000000
30%         399.000000
40%         411.000000
50%         422.000000
60%         434.000000
70%         447.000000
80%         463.000000
90%         482.000000
95%         491.000000
99%         500.000000
max         502.000000
Name: context_token_length, dtype: float64

In [35]:
df.sample(20).head(20)

,Id,is_multi,ground_truth,dataset_labels,is_impossible,answer_start,answer_end,context,context_token_length
280982,fd4203de-fbf8-49ef-bbd4-430ec454cd68,1,coastal change analysis program|noaa c cap,Coastal Change Analysis Program|NOAA C-CAP,1,-1,-1,"mats could be found on eelgrass beds and bare substrate without distinguishable spectral variation, creating a broader range of spectral signatures found within the bay. Furthermore, there are two species of eelgrass that thrive in Willapa Bay, each with a unique signature. North American eelgrass (Zostera Marina) is larger leafed than Asian eelgrass (Zostera Japonica) and tends to occupy the inter-tidal zone (Backman, 1991;Phillips, 1984). The lighter hue of Asian eelgrass could easily be detected on the tidal flats, but could not be clearly distinguished from North American eelgrass in the inter-tidal zone. The combination of the two eelgrass species in the inter-tidal zone provided a unique spectral signature of its own, leading to three different signatures of eelgrass. There were other unidentifiable spectral signatures for which no source could be located. Several regions in the northern end of the study area had received sediment deposition following the flight mission and consequently no source for reflectance could be determined. The use of preliminary data gathered to measure spectral signatures was critical for the accurate interpretation of the Willapa Bay eelgrass photo set and will prove to be an integral component to future eelgrass distribution studies in Willapa Bay. Determining the size of the minimum mapping unit for this study proved to be problematic due to the presence of a non-native rush, Spartina spp., throughout Willapa Bay. C-CAP guidelines suggest that a minimum mapping unit of .03 hectares on the ground is appropriate for SRV distribution studies under ideal conditions. However, the prolific growth of Spartina spp., throughout Willapa Bay created a situation where countless island polygons of Spartina spp., clones would need to be excluded from eelgrass beds if the .03 hectare minimum mapping unit was used. To avoid mapping Spartina spp. rather than eelgrass, the minimum mapping unit size was increased to 1/10"" or 2.54 millimeters on the",407
320698,76d0526a-4dd6-478e-bd6e-d0f331e41cd3,0,slosh model,SLOSH model,1,-1,-1,"y in Moses Bayou is also impaired from bacteria, dioxin in edible tissue, and PCBs in edible tissue. Total maximum daily loads (TMDLs) in Moses Lake and Bayou for dioxin in edible tissue and PCBs in edible tissue are planned. Additional data is being collected before a management strategy is selected for the bacteria impairment.\nThere are two major tidal inlets into Corpus Christi Bay. Aransas Pass (Corpus Christi Ship Channel), between Mustang Island and San Jose Island, which accounts for the majority of the tidal exchange between the bay and the Gulf of Mexico. Packery Channel, between the southwestern end of Mustang Island and North Padre Island, is manmade inlet that supplies a lesser amount of the bay's tidal exchange. Overall, the natural depth of the bay is relatively shallow, with an average depth of approximately 9 feet. Tides in Corpus Christi Bay under normal conditions are very small in amplitude, usually less than 3 feet between low and high tide. Wind speed and direction within Corpus Christi Bay plays an important role in affecting tide elevation. It can dampen or enhance the height of waves as well as their potential energy. Prevailing winds are from the southeast, with occasional strong northerly winds that are associated with passing cold fronts. Winds combined with seasonal tide events can greatly exacerbate the tidal range as well as move the range up or down by 1 or 2 feet. Storm tides during Category 4 or 5 hurricanes could be as high as 15-20 feet above normal water levels according to NOAA's Sea, Lake, and Overland Surge from Hurricanes (SLOSH) Model. The project site is located along Indian Point in Upper Corpu

In [36]:
df.to_parquet("output/train.parquet", index=False)